In [1]:
import numpy as np
import pandas as pd
from coxdev import (CoxDeviance,
                    _reversed_cumsums)

from simulate import (simulate_df, 
                      all_combos,
                      rng)

In [19]:
data = simulate_df(all_combos[-1], nrep=3, size=4)
data = data.reset_index().drop(columns='index')
data

,start,event,status
0,1.301881,1.830246,1
1,1.423530,2.642750,1
2,0.766130,1.282122,0
3,4.038934,8.296885,0
4,0.942753,1.955512,0
...,...,...,...
167,2.503371,3.173360,0
168,2.523544,3.173360,0
169,3.173360,3.839091,0
170,3.173360,6.480528,0


In [20]:
cox = CoxDeviance(event=data['event'],
                  start=data['start'],
                  status=data['status'],
                  tie_breaking='efron')
(data['event'] < data['start']).sum()

0

In [21]:
X = rng.standard_normal(data.shape[0])
(X_event, 
 X_start) = _reversed_cumsums(X, 
                              event_order=cox._event_order,
                              start_order=cox._start_order)

tmp = X_event[cox._first] - X_start[cox._event_map]
cumsum_diff = np.zeros_like(tmp)
cumsum_diff[cox._event_order] = tmp

In [23]:
by_hand = []
for i in range(data.shape[0]):
    val = X[(data['event'] >= data['event'].iloc[i]) & (data['start'] < data['event'].iloc[i])].sum()
    by_hand.append(val)
by_hand = np.array(by_hand)
assert np.allclose(by_hand * np.array(data['status']), cumsum_diff * np.array(data['status']))

AssertionError: 